# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor usage and billing.


In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
file_path = ("../WeatherPy/output_data/cities.csv")
weather_df = pd.read_csv(file_path)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key = g_key)

In [19]:
figure_layout = {'width': '990px',
                'height': '600px',
                'border': '1px solid black',
                'padding': '1px',
                'margin': '1 auto 0 auto'
                }

fig = gmaps.figure(layout=figure_layout, center=(20,0), zoom_level= 2)

coordinates = weather_df[["Lat" , "Lng"]].sort_values(by = ["Lng"])

weight = weather_df["Humidity"].astype(float)
heat_layer = gmaps.heatmap_layer(coordinates, weights = weight, dissipating= False, max_intensity = 100 , point_radius = 3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='1 auto 0 auto', padding='1px', wi…

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='1 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
weather_df.describe()

,City_ID,Cloudiness,Datetime,Humidity,Lat,Lng,Max Temp,Wind Speed
count,566.000000,566.000000,5.660000e+02,566.000000,566.000000,566.000000,566.000000,566.000000
mean,304.653710,60.814488,1.639907e+09,72.614841,20.279056,15.870078,10.951272,3.499912
std,177.334653,38.275192,3.530415e+02,22.095851,32.782491,90.314660,17.883874,2.636987
min,0.000000,0.000000,1.639906e+09,8.000000,-54.800000,-175.200000,-46.010000,0.000000
25%,148.250000,20.000000,1.639907e+09,63.000000,-6.824475,-61.610550,-0.140000,1.540000
50%,308.500000,75.000000,1.639907e+09,78.000000,22.839750,19.505250,17.405000,2.875000
75%,455.750000,98.000000,1.639907e+09,89.000000,47.976925,92.246025,24.952500,4.715000
max,611.000000,100.000000,1.639908e+09,100.000000,78.218600,178.416700,39.750000,16.460000


,City_ID,Cloudiness,Datetime,Humidity,Lat,Lng,Max Temp,Wind Speed
count,566.000000,566.000000,5.660000e+02,566.000000,566.000000,566.000000,566.000000,566.000000
mean,304.653710,60.814488,1.639907e+09,72.614841,20.279056,15.870078,10.951272,3.499912
std,177.334653,38.275192,3.530415e+02,22.095851,32.782491,90.314660,17.883874,2.636987
min,0.000000,0.000000,1.639906e+09,8.000000,-54.800000,-175.200000,-46.010000,0.000000
25%,148.250000,20.000000,1.639907e+09,63.000000,-6.824475,-61.610550,-0.140000,1.540000
50%,308.500000,75.000000,1.639907e+09,78.000000,22.839750,19.505250,17.405000,2.875000
75%,455.750000,98.000000,1.639907e+09,89.000000,47.976925,92.246025,24.952500,4.715000
max,611.000000,100.000000,1.639908e+09,100.000000,78.218600,178.416700,39.750000,16.460000


In [21]:
# narrow down the cities list based on these criteria:
#     1. Wind speed less then 5 mph
#     2. Temperature is between 20 and 26 celsius
#     3. No cloud
hotel_df = weather_df.loc[(weather_df["Wind Speed"] < 5)&
                              (weather_df["Max Temp"] > 20)&
                             (weather_df["Max Temp"] < 26)&
                             (weather_df["Cloudiness"] == 0)
                             ]
print(f"Narrow down to {len(hotel_df)} from {len(weather_df)} cities")
hotel_df.reset_index()

Narrow down to 9 from 566 cities


,index,City_ID,City,Cloudiness,Country,Datetime,Humidity,Lat,Lng,Max Temp,Wind Speed
0,36,37,baihar,0,IN,1639906543,12,22.1000,80.5500,21.88,2.63
1,225,245,maceio,0,BR,1639906939,94,-9.6658,-35.7353,24.69,2.06
2,229,250,nueva germania,0,PY,1639907003,72,-23.9000,-56.5667,25.84,2.09
3,320,348,shahpura,0,IN,1639907137,11,27.3833,75.9667,20.63,3.48
4,341,369,namibe,0,AO,1639907212,68,-15.1961,12.1522,23.35,3.60
5,377,407,cordoba,0,AR,1639907126,76,-31.4135,-64.1811,23.20,0.45
6,390,420,san luis,0,AR,1639907309,47,-33.2950,-66.3356,24.37,3.40
7,465,501,naraina,0,IN,1639907488,9,26.7833,75.2000,21.08,1.33
8,469,506,tura,0,IN,1639907492,52,25.5198,90.2201,20.72,2.33


Narrow down to 9 from 566 cities


,index,City_ID,City,Cloudiness,Country,Datetime,Humidity,Lat,Lng,Max Temp,Wind Speed
0,36,37,baihar,0,IN,1639906543,12,22.1000,80.5500,21.88,2.63
1,225,245,maceio,0,BR,1639906939,94,-9.6658,-35.7353,24.69,2.06
2,229,250,nueva germania,0,PY,1639907003,72,-23.9000,-56.5667,25.84,2.09
3,320,348,shahpura,0,IN,1639907137,11,27.3833,75.9667,20.63,3.48
4,341,369,namibe,0,AO,1639907212,68,-15.1961,12.1522,23.35,3.60
5,377,407,cordoba,0,AR,1639907126,76,-31.4135,-64.1811,23.20,0.45
6,390,420,san luis,0,AR,1639907309,47,-33.2950,-66.3356,24.37,3.40
7,465,501,naraina,0,IN,1639907488,9,26.7833,75.2000,21.08,1.33
8,469,506,tura,0,IN,1639907492,52,25.5198,90.2201,20.72,2.33


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = hotel_df[["City", "Country", "Lat", "Lng"]]

params = {"keyword": "hotel",
          "radius": 5000,
          "type": "lodging",
          "key": g_key}

hotel_df["Hotel Name"] = ""

In [23]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
36,baihar,IN,22.1000,80.5500,
225,maceio,BR,-9.6658,-35.7353,
229,nueva germania,PY,-23.9000,-56.5667,
320,shahpura,IN,27.3833,75.9667,
341,namibe,AO,-15.1961,12.1522,
377,cordoba,AR,-31.4135,-64.1811,
390,san luis,AR,-33.2950,-66.3356,
465,naraina,IN,26.7833,75.2000,
469,tura,IN,25.5198,90.2201,


,City,Country,Lat,Lng,Hotel Name
36,baihar,IN,22.1000,80.5500,
225,maceio,BR,-9.6658,-35.7353,
229,nueva germania,PY,-23.9000,-56.5667,
320,shahpura,IN,27.3833,75.9667,
341,namibe,AO,-15.1961,12.1522,
377,cordoba,AR,-31.4135,-64.1811,
390,san luis,AR,-33.2950,-66.3356,
465,naraina,IN,26.7833,75.2000,
469,tura,IN,25.5198,90.2201,


In [24]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# loop through DataFrame to find a hotel within 5kms of each city's coordinate 
for index, row in hotel_df.iterrows():
    found = False
    coordinates = f"{row['Lat']}, {row['Lng']}"
    params["location"] = coordinates
    city = hotel_df.loc[index, 'City'].title()
    
    try:
        response = requests.get(base_url, params).json()
        print(f"Found {len(response['results'])} lodging(s) within 5 kms from {city}'s coordinate: ")

        # loop through lodgings found and search for hotel in the name
        for lodging in response["results"]:
            lodging_name = lodging["name"]
            print(f"    {lodging_name}", end = " ")
            if "hotel" in lodging["name"].lower():
                print("Found! \n")
                hotel_df.loc[index , "Hotel Name"] = lodging_name 
                found = True
                break
            else:
                print("is not a hotel!")
        # if no hotel found:        
        if found == False:
            print(f"    No hotel found in {city} \n")
            hotel_df.loc[index , "Hotel Name"] = "Hotel NOT FOUND!"
            
    except (KeyError,IndexError):
        pass
        print("Missing field/result... skipping.")
        
    print("----------------------------------")

Found 0 lodging(s) within 5 kms from Baihar's coordinate: 
    No hotel found in Baihar 

----------------------------------
Found 0 lodging(s) within 5 kms from Baihar's coordinate: 
    No hotel found in Baihar 

----------------------------------
Found 0 lodging(s) within 5 kms from Maceio's coordinate: 
    No hotel found in Maceio 

----------------------------------
Found 0 lodging(s) within 5 kms from Maceio's coordinate: 
    No hotel found in Maceio 

----------------------------------
Found 0 lodging(s) within 5 kms from Nueva Germania's coordinate: 
    No hotel found in Nueva Germania 

----------------------------------
Found 0 lodging(s) within 5 kms from Nueva Germania's coordinate: 
    No hotel found in Nueva Germania 

----------------------------------
Found 0 lodging(s) within 5 kms from Shahpura's coordinate: 
    No hotel found in Shahpura 

----------------------------------
Found 0 lodging(s) within 5 kms from Shahpura's coordinate: 
    No hotel found in Shahpu

In [25]:
# Save DataFrame
hotel_df.to_csv("output_data/hotels.csv", index = False)

In [26]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
36,baihar,IN,22.1000,80.5500,Hotel NOT FOUND!
225,maceio,BR,-9.6658,-35.7353,Hotel NOT FOUND!
229,nueva germania,PY,-23.9000,-56.5667,Hotel NOT FOUND!
320,shahpura,IN,27.3833,75.9667,Hotel NOT FOUND!
341,namibe,AO,-15.1961,12.1522,Hotel NOT FOUND!
377,cordoba,AR,-31.4135,-64.1811,Hotel NOT FOUND!
390,san luis,AR,-33.2950,-66.3356,Hotel NOT FOUND!
465,naraina,IN,26.7833,75.2000,Hotel NOT FOUND!
469,tura,IN,25.5198,90.2201,Hotel NOT FOUND!


,City,Country,Lat,Lng,Hotel Name
36,baihar,IN,22.1000,80.5500,Hotel NOT FOUND!
225,maceio,BR,-9.6658,-35.7353,Hotel NOT FOUND!
229,nueva germania,PY,-23.9000,-56.5667,Hotel NOT FOUND!
320,shahpura,IN,27.3833,75.9667,Hotel NOT FOUND!
341,namibe,AO,-15.1961,12.1522,Hotel NOT FOUND!
377,cordoba,AR,-31.4135,-64.1811,Hotel NOT FOUND!
390,san luis,AR,-33.2950,-66.3356,Hotel NOT FOUND!
465,naraina,IN,26.7833,75.2000,Hotel NOT FOUND!
469,tura,IN,25.5198,90.2201,Hotel NOT FOUND!


In [27]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='1 auto 0 auto', padding='1px', wi…

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='1 auto 0 auto', padding='1px', wi…